In [2]:
import requests
import pandas as pd
import json
import time

In [ ]:
#variables
client_id = 'f4137175f2274f52b5a5a0038f515e2b'
client_secret = ''
token_url = 'https://oauth.battle.net/token'


region = 'us'
seasons = [34, 35, 36, 37] 
pvpBracket = '3v3'
namespace = 'dynamic-us'
locale = 'en_US'


In [4]:
#get access token

# Request payload
data = {
    'grant_type': 'client_credentials'
}

# Basic authentication
auth = (client_id, client_secret)

# Make the request to get the access token
response = requests.post(token_url, data=data, auth=auth)

if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print(f"Access Token: {access_token}")
else:
    print(f"Error {response.status_code}: {response.text}")

Access Token: USODxkYOarYwjC4IsISKJy6H63FJRyc4pD


In [ ]:
#call WOW pvp ranking API for each season and get PVP ranking and character
def get_pvp_ranking(region, seasons, pvpBracket, namespace, locale, access_token):
    global leaderboard_df
    all_data = []
    
    for pvpSeasonId in seasons:
        url = f'https://{region}.api.blizzard.com/data/wow/pvp-season/{pvpSeasonId}/pvp-leaderboard/{pvpBracket}'
        params = {
            'namespace': namespace,
            'locale': locale,
            'access_token': access_token
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            entries = data.get('entries', [])
            if entries:
                leaderboard_df_new = pd.DataFrame.from_dict(pd.json_normalize(entries), orient='columns')
                leaderboard_df_new['pvpSeason'] = pvpSeasonId
                all_data.append(leaderboard_df_new)
        else:
            print(f"Error {response.status_code} for season {pvpSeasonId}: {response.text}")

    # Concatenate all DataFrames into a single DataFrame
    leaderboard_df = pd.concat(all_data, ignore_index=True)
    return leaderboard_df

In [ ]:
get_pvp_ranking(region, seasons, pvpBracket, namespace, locale, access_token)
leaderboard_df.info()

,rank,rating,character.name,character.id,character.realm.key.href,character.realm.id,character.realm.slug,faction.type,season_match_statistics.played,season_match_statistics.won,season_match_statistics.lost,tier.key.href,tier.id,pvpSeason
0,1,3129,Nanoir,226349771,https://us.api.blizzard.com/data/wow/realm/57?...,57,illidan,ALLIANCE,208,131,77,https://us.api.blizzard.com/data/wow/pvp-tier/...,14,34
1,2,3124,Manofmystery,150980905,https://us.api.blizzard.com/data/wow/realm/137...,1377,drenden,ALLIANCE,248,174,74,https://us.api.blizzard.com/data/wow/pvp-tier/...,14,34
2,2,3124,Ryuzu,154118499,https://us.api.blizzard.com/data/wow/realm/116...,1165,arathor,HORDE,240,162,78,https://us.api.blizzard.com/data/wow/pvp-tier/...,14,34
3,4,3115,Dronedròp,226295766,https://us.api.blizzard.com/data/wow/realm/57?...,57,illidan,HORDE,524,323,201,https://us.api.blizzard.com/data/wow/pvp-tier/...,14,34
4,5,3109,Askronnie,194611442,https://us.api.blizzard.com/data/wow/realm/76?...,76,sargeras,ALLIANCE,1014,593,421,https://us.api.blizzard.com/data/wow/pvp-tier/...,14,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20050,5000,2202,Xiaoguo,242105071,https://us.api.blizzard.com/data/wow/realm/11?...,11,tichondrius,HORDE,562,284,278,https://us.api.blizzard.com/data/wow/pvp-tier/...,13,37
20051,5000,2202,Startx,238355628,https://us.api.blizzard.com/data/wow/realm/11?...,11,tichondrius,ALLIANCE,261,131,130,https://us.api.blizzard.com/data/wow/pvp-tier/...,13,37
20052,5000,2202,Aleistria,148370185,https://us.api.blizzard.com/data/wow/realm/162...,162,emerald-dream,ALLIANCE,420,209,211,https://us.api.blizzard.com/data/wow/pvp-tier/...,13,37
20053,5000,2202,Osmatspriest,174458076,https://us.api.blizzard.com/data/wow/realm/127...,127,firetree,ALLIANCE,438,215,223,https://us.api.blizzard.com/data/wow/pvp-tier/...,13,37


In [ ]:
#Call character API to look up character class/spec data points 

def get_spec(characterName, realmSlug):


    char_url = f'https://us.api.blizzard.com/profile/wow/character//{realmSlug}/{characterName}'

    char_params = {
        'namespace': 'profile-us',
        'locale': 'en_US',
        'access_token': access_token
    }

    char_response = requests.get(char_url, params=char_params)

    if char_response.status_code == 200:
        char_data = char_response.json()
        spec_name =char_data['active_spec']['name'] if 'active_spec' in char_data else f"None"
        class_name = char_data['character_class']['name'] if 'character_class' in char_data else f"None"
        return(spec_name+' '+class_name)
    else:
        return f"Not Found"


In [ ]:
#save to json file
leaderboard_df.to_json(r"C:\Users\Joun\Documents\Data projects\wowpvpdata.json", orient='records', indent=4)

print('DataFrame is written to json File successfully.')


DataFrame is written to json File successfully.
